In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import glob
import os
import re
from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from IPython.display import display, HTML


from aind_dynamic_foraging_data_utils import nwb_utils as nu
import aind_dynamic_foraging_basic_analysis.licks.annotation as a

from aind_dynamic_foraging_basic_analysis import plot_foraging_session, compute_foraging_efficiency

In [41]:
data_dir = ''

nwb = nu.load_nwb_from_filename('/root/capsule/data/foraging_nwb_bonsai/687582_2023-11-16_12-13-24.nwb')

nwb.df_trials = nu.create_df_trials(nwb)

nwb.df_trials

/opt/conda/lib/python3.9/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.1 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/opt/conda/lib/python3.9/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.4.0 because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/opt/conda/lib/python3.9/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.2.0 because version 0.5.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


Timestamps are adjusted such that `_in_session` timestamps start at the first go cue


/opt/conda/lib/python3.9/site-packages/aind_dynamic_foraging_data_utils/nwb_utils.py:457: UserWarning: Reward before choice time. This is likely due to manual rewards
  warnings.warn("Reward before choice time. This is likely due to manual rewards")


,trial,animal_response,rewarded_historyL,rewarded_historyR,bait_left,bait_right,base_reward_probability_sum,reward_probabilityL,reward_probabilityR,reward_random_number_left,...,goCue_start_time_in_trial,reward_outcome_time_in_session,reward_outcome_time_in_trial,goCue_start_time_raw,reward_time_in_session,reward_time_in_trial,choice_time_in_session,choice_time_in_trial,earned_reward,extra_reward
0,0,2.0,False,False,False,False,0.8,0.711111,0.088889,0.963379,...,0.0,2.744864,2.744864,4.489579e+06,NaN,NaN,NaN,NaN,False,False
1,1,2.0,False,False,True,False,0.8,0.711111,0.088889,0.526161,...,0.0,11.984864,3.013856,4.489588e+06,NaN,NaN,NaN,NaN,False,False
2,2,2.0,False,False,True,False,0.8,0.711111,0.088889,0.904736,...,0.0,22.649856,3.013856,4.489599e+06,NaN,NaN,NaN,NaN,False,False
3,3,2.0,False,False,True,False,0.8,0.711111,0.088889,0.758493,...,0.0,33.501856,3.015840,4.489610e+06,NaN,NaN,NaN,NaN,False,False
4,4,2.0,False,False,True,False,0.8,0.711111,0.088889,0.555699,...,0.0,45.584864,3.013856,4.489622e+06,NaN,NaN,NaN,NaN,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,537,2.0,False,False,True,False,0.8,0.711111,0.088889,0.608060,...,0.0,5359.988864,3.013792,4.494936e+06,NaN,NaN,NaN,NaN,False,False
538,538,2.0,False,False,True,False,0.8,0.711111,0.088889,0.030398,...,0.0,5371.121856,3.013792,4.494947e+06,NaN,NaN,NaN,NaN,False,False
539,539,2.0,False,False,True,False,0.8,0.711111,0.088889,0.726525,...,0.0,5382.621856,3.013760,4.494959e+06,NaN,NaN,NaN,NaN,False,False
540,540,2.0,False,False,True,True,0.8,0.711111,0.088889,0.587486,...,0.0,5391.454880,3.013792,4.494968e+06,5388.442368,0.00128,NaN,NaN,False,True


In [93]:
x = nwb.df_trials['animal_response'] == 2
metric = 1 - np.mean(x)

print(metric)

0.4446494464944649


In [94]:
def session_metrics(nwb):

    if not hasattr(nwb, 'df_trials'):
        print('You need to compute df_trials: nwb_utils.create_trials_df(nwb)')
        return
    
    # METADATA PLACEHOLDER 
    session_start_time = nwb.session_start_time
    session_date = session_start_time.strftime("%Y-%m-%d")
    subject_id_from_meta = nwb.subject.subject_id
    
    # Parse the file name for suffix
    old_re = re.match(r"(?P<subject_id>\d+)_(?P<date>\d{4}-\d{2}-\d{2})(?:_(?P<n>\d+))?\.json", 
                nwb.session_id)
    
    if old_re is not None:
        subject_id, session_date, nwb_suffix = old_re.groups()
        nwb_suffix = int(nwb_suffix) if nwb_suffix is not None else 0
    else:
        subject_id, session_date, session_json_time = re.match(r"(?P<subject_id>\d+)_(?P<date>\d{4}-\d{2}-\d{2})(?:_(?P<time>.*))\.json", 
                            nwb.session_id).groups()
        nwb_suffix = int(session_json_time.replace('-', ''))
    
    # Verify metadata matches
    assert subject_id == subject_id_from_meta, f"Subject name from the metadata ({subject_id_from_meta}) does not match "\
                                               f"that from json name ({subject_id})!!"
    assert session_date == session_date, f"Session date from the metadata ({session_date}) does not match "\
                                                   f"that from json name ({session_date})!!"
    
    # Create session index
    session_index = pd.MultiIndex.from_tuples([(subject_id, session_date, nwb_suffix)], 
                                            names=['subject_id', 'session_date', 'nwb_suffix'])

    # Get metadata from nwb.scratch
    meta_dict = nwb.scratch['metadata'].to_dataframe().iloc[0].to_dict()
    
    # Calculate performance metrics
    n_total_trials = len(df_trial)
    n_finished_trials = (df_trial.animal_response != IGNORE).sum()
    
    # Actual foraging trials (autowater excluded)
    n_total_trials_non_autowater = df_trial.non_autowater_trial.sum()
    n_finished_trials_non_autowater = df_trial.non_autowater_finished_trial.sum()
    
    n_reward_trials_non_autowater = df_trial.reward_non_autowater.sum()
    reward_rate_non_autowater_finished = n_reward_trials_non_autowater / n_finished_trials_non_autowater if n_finished_trials_non_autowater > 0 else np.nan
    
    # Calculate foraging efficiency
    foraging_eff, foraging_eff_random_seed = compute_foraging_efficiency(
        baited='without bait' not in nwb.protocol.lower(),
        choice_history=df_trial.animal_response.map({0: 0, 1: 1, 2: np.nan}).values,
        reward_history=df_trial.rewarded_historyL | df_trial.rewarded_historyR,
        p_reward=[
            df_trial.reward_probabilityL.values,
            df_trial.reward_probabilityR.values,
        ],
        random_number=[
            df_trial.reward_random_number_left.values,
            df_trial.reward_random_number_right.values,
        ],
        autowater_offered=(df_trial.auto_waterL == 1) | (df_trial.auto_waterR == 1),
    )
    
    # Override foraging_eff_random_seed for old bpod sessions
    if 'bpod' in nwb.session_description:
        foraging_eff_random_seed = nwb.get_scratch('metadata')['foraging_efficiency_with_actual_random_seed'].values[0]
    
    finished_rate = n_finished_trials_non_autowater / n_total_trials_non_autowater if n_total_trials_non_autowater > 0 else np.nan
    
    # New Metrics

    # Probability chosen calculation 
    probability_chosen = []
    probability_not_chosen = []

    for _, row in df_trial.iterrows():
        if row.animal_response == 2:
            probability_chosen.append(np.nan)
            probability_not_chosen.append(np.nan)
        elif row.animal_response == 0:  # Chosen = left choice left probability, not chosen = left choice right probability 
            probability_chosen.append(row.reward_probabilityL)
            probability_not_chosen.append(row.reward_probabilityR)
        else: # Chosen = right choice right probability, not chosen = right choice left probability 
            probability_chosen.append(row.reward_probabilityR)
            probability_not_chosen.append(row.reward_probabilityL)

    df_trial['probability_chosen'] = probability_chosen
    df_trial['probability_not_chosen'] = probability_not_chosen

    # Calculate the chosen probability
    average = (df_trial['probability_chosen'] - df_trial['probability_not_chosen'])
    
    p_larger_global = max(df_trial['probability_chosen'].max(), df_trial['probability_not_chosen'].max())
    
    p_smaller_global = min(df_trial['probability_chosen'].min(), df_trial['probability_not_chosen'].min())
    
    mean_difference = average.mean()
    chosen_probability = mean_difference / (p_larger_global - p_smaller_global)


    # Pack all data
    dict_meta = {
        # Basic metadata
        'rig': meta_dict['box'],        
        'user_name': nwb.experimenter[0],
        'task': nwb.protocol,
        
        # New metric
        'chosen_probability': chosen_probability,

        # Trial counts and rates
        'total_trials': n_total_trials_non_autowater,
        'finished_trials': n_finished_trials_non_autowater,
        'finished_rate': finished_rate,
        'total_trials_with_autowater': n_total_trials,
        'finished_trials_with_autowater': n_finished_trials,
        'finished_rate_with_autowater': n_finished_trials / n_total_trials,
        
        # Reward and foraging metrics
        'reward_trials': n_reward_trials_non_autowater,
        'reward_rate': reward_rate_non_autowater_finished,
        'foraging_eff': foraging_eff,
        'foraging_eff_random_seed': foraging_eff_random_seed,
        'foraging_performance': foraging_eff * finished_rate,
        'foraging_performance_random_seed': foraging_eff_random_seed * finished_rate,
        
        # Timing metrics
        'reaction_time_median': df_trial.loc[:, 'reaction_time'].median(),
        'reaction_time_mean': df_trial.loc[:, 'reaction_time'].mean(),
        'early_lick_rate': (df_trial.loc[:, 'n_lick_all_delay_period'] > 0).sum() / n_total_trials,
        
        # Double dipping metrics
        'double_dipping_rate_finished_trials': 
            (df_trial.loc[(df_trial.animal_response != IGNORE), 'n_lick_switches_gocue_stop'] > 0).sum() 
            / (df_trial.animal_response != IGNORE).sum(),
        'double_dipping_rate_finished_reward_trials':
            (df_trial.loc[df_trial.reward, 'n_lick_switches_gocue_stop'] > 0).sum()  
            / df_trial.reward.sum(),
        'double_dipping_rate_finished_noreward_trials':
            (df_trial.loc[(df_trial.animal_response != IGNORE) & (~df_trial.reward), 'n_lick_switches_gocue_stop'] > 0).sum() 
            / ((df_trial.animal_response != IGNORE) & (~df_trial.reward)).sum(),
            
        # Lick consistency metrics
        'lick_consistency_mean_finished_trials': 
            df_trial.loc[(df_trial.animal_response != IGNORE), 'n_lick_consistency_gocue_stop'].mean(),
        'lick_consistency_mean_finished_reward_trials': 
            df_trial.loc[df_trial.reward, 'n_lick_consistency_gocue_stop'].mean(),
        'lick_consistency_mean_finished_noreward_trials': 
            df_trial.loc[(df_trial.animal_response != IGNORE) & (~df_trial.reward), 'n_lick_consistency_gocue_stop'].mean(),
    }
    
    # Create DataFrame with hierarchical columns
    df_meta = pd.DataFrame(dict_meta, 
                          index=session_index)
    df_meta.columns = pd.MultiIndex.from_product([['metadata'], dict_meta.keys()],
                                                names=['type', 'variable'])
    
    return df_meta

In [95]:
test_df = session_metrics(nwb)

display(HTML(test_df.to_html()))

In [32]:
final_df = compute_df_session_performance(nwb, df_trial)

/opt/conda/lib/python3.9/site-packages/aind_dynamic_foraging_basic_analysis/foraging_efficiency.py:107: RuntimeWarning: divide by zero encountered in scalar divide
  reward_actual / reward_optimal_random_seed,


In [73]:
display(HTML(final_df.to_html()))